# Práctica 1: Perceptrón multicapa.

Tu jefe pidió a RH que recolectara datos de desempeño de tus compañeros, los resultados se almacenaron en un csv. El punto critico de estos datos es la satisfacción del empleado, entonces ¿Podremos estimar la satisfacción de los empleados con los datos recabados?.

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models


df = pd.read_csv('Extended_Employee_Performance_and_Productivity_Data.csv')
df.info()

ModuleNotFoundError: No module named 'tensorflow.python'

In [ ]:
# Filtrar las columnas numéricas
numeric_columns = df.select_dtypes(include=['number']).drop('Employee_ID',axis=1)


# Si numeric_columns es un Index, conviértelo a lista
cols = list(numeric_columns)

fig, axes = plt.subplots(1, len(cols), figsize=(5 * len(cols), 4))

for i, col in enumerate(cols):
    axes[i].hist(df[col], bins=20, color='skyblue', edgecolor='black')
    axes[i].set_title(col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

**Problemas**, tenemos distribuciones con picos, esos nos indica categorías. Por otro lado, tenemos variables con "valles" en su distribución (distribuciones multimodales) por lo que resultaría óptimo aplicar técnicas de feature engeneering. Por último tenemos distribuciones uniformes, por lo que cada una requeriría un procesamiento indivudual, hagamos la vista gorda e intentemos ajustar un MLP con estos datos, solo estandaricemos nuestros datos.

---

## Implementación de Red:

To**memos los datos numéricos como nuestra variable X, y la variable objetivo como ***'Employee_Satisfaction_Score'***.
- **Actividad 1**: Para todos los strings ``'@modif@'`` que aparescan en el siguiente bloque de código cámbialos para que el código funcione.

In [ ]:
X = numeric_columns.drop('Employee_Satisfaction_Score', axis=1)
y = numeric_columns['Employee_Satisfaction_Score']

y = y.apply(lambda x: round(x) - 1)

scaler = StandardScaler()
X_standar = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_standar, y, test_size=0.33, random_state=42)

y_onehot_train = tf.keras.utils.to_categorical(y_train, 5)
y_onehot_test = tf.keras.utils.to_categorical(y_test, 5)

- **Actividad 2:** Implementa 3 arquitecturas de MLP, cada una con su propio nombre, cambiando la estructura de dichas arquitecturas (capas, neuronas por capa, función de activación, etc). 

In [ ]:
# Modelo 1: Simple
model1 = models.Sequential(name="MLP_simple")
model1.add(layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model1.add(layers.Dense(16, activation='relu'))
model1.add(layers.Dense(5, activation='softmax'))

# Modelo 2: Medio con dropout
model2 = models.Sequential(name="MLP_dropout")
model2.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model2.add(layers.Dropout(0.3))
model2.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(5, activation='softmax'))

# Modelo 3: Profundo
model3 = models.Sequential(name="MLP_deep")
model3.add(layers.Dense(128, activation='tanh', input_shape=(X_train.shape[1],)))
model3.add(layers.Dense(64, activation='tanh'))
model3.add(layers.Dense(32, activation='relu'))
model3.add(layers.Dense(16, activation='relu'))
model3.add(layers.Dense(5, activation='softmax'))

- **Actividad 3:** Compila y ajusta tus tres modelos con sus respectivos hiperparámetros.

In [ ]:
#Configuración común
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

for model in [model1, model2, model3]:
    print(f"\nEntrenando {model.name} ...")
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    history = model.fit(X_train, y_onehot_train,
                        epochs=30,
                        batch_size=32,
                        validation_data=(X_test, y_onehot_test),
                        verbose=1)
    
    # Evaluación final
    loss, acc = model.evaluate(X_test, y_onehot_test, verbose=0)
    print(f"Modelo {model.name} → Test Accuracy: {acc:.4f}")

- **Actividad 4:** Sube tus cambios al repositorio, envía el link de tu repositorio a la actividad 2 de tu checkpoint 2 y contesta las preguntas de dicha actividad.

Si los datos son moderadamente grandes y complejos, yo me quedaría con MLP_dropout.